# This is an apporach to predict and verify the WSD based on the sense tags.
- GPT 3.5 is used as the parent model for prediction, while same model is used to verify the sense tags.
- verification is based on Zero shot prompting technique

- Steps to follow

1. Identify the senses/glosses based on the POS tag and the synonyms passsed through the query.
2. Iterative mechanism is used to identify the sense/gloss based on the approach
3. Zer Shot prompting is used to verify the sense/gloss based on the approach
4. If the verification is same then the sense/gloss is identified else the sense/gloss will iterate the predicting process by reducing the definition space by 1.
4. Process continues until the sense/gloss is identified. if the sense/gloss is not identified then the sense/gloss will be identified as "None"

In [27]:
import os, yaml, re
import json
from openai import OpenAI, ChatCompletion

In [3]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']



In [4]:
client = OpenAI()

In [5]:
def complete_model(USER_MESSAGE):
    response = client.chat.completions.create(
                                            model = 'gpt-3.5-turbo',
                                            messages = [
                                                        {"role": "system", "content" : "You are a helpful assitant to identify the tag for sense of a word"},
                                                        {"role": "user", "content": USER_MESSAGE}              
                                                        ],
                                            temperature=0,
                                            max_tokens=1500
                                            )
    return str(response.choices[0].message.content)

In [6]:
# Read data from file
with open('senses.txt', 'r',encoding="utf8") as file:
    data = file.read()

# Split the data into entries based on empty lines
entries = data.strip().split('\n\n')
print("No of sense tags in FEWS dataset ",len(entries))

# Create a list of lists for each entry's details
list_of_lists = []
for entry in entries:
    details = entry.split('\n')
    entry_list = []
    for detail in details:
        _, value = detail.split(':', 1)
        entry_list.append(value.strip())
    list_of_lists.append(entry_list)

# Print the instance from the sense tag
print(list_of_lists[0])

No of sense tags in FEWS dataset  663730
['dictionary.noun.0', 'dictionary', "A reference work with a list of words from one or more languages, normally ordered alphabetically, explaining each word's meaning, and sometimes containing information on its etymology, pronunciation, usage, translations, and other data.", '', '1', 'wordbook']


In [7]:
#converting the word into the base word as the sense are kept in base
def word_base(word):
    try:
        word=stemmer.stem(word)
        base_word = lemmatizer.lemmatize(word)
        return base_word
    except:
        return word   
    

In [8]:
#function to retrieve the word meaning from the list_of_list list
#this function will specifically read the sense id and the meaning(gloss) which is required for the processing.
def retrieve_meanings(word, data):
    meanings_dict = {}
    for entry in data:
        if word == entry[0].split(".")[0]:
            if entry[-1] !="":
                meanings_dict[entry[0]] = entry[2]+", synonyms :"+entry[-1]
            else:
                meanings_dict[entry[0]] = entry[2]
    return meanings_dict

In [9]:
#Extract the sentence without <wsd> token and the index
def extract_word_and_index(sentence):
    # Find the start and end index of the <WSD> tags
    start_index = sentence.find('<WSD>')    
    end_index = sentence.find('</WSD>')
    

    if start_index != -1 and end_index != -1:
        # Extract the word between <WSD> tags
        word = sentence[start_index + len('<WSD>'):end_index]

        # Remove <WSD> and </WSD> tags from the sentence
        cleaned_sentence = sentence[:start_index] + word+" " + sentence[end_index + len('</WSD>'):]
            
        return cleaned_sentence.strip()

In [10]:
#Function to identify the WSD word from the given sentence and return the WSD word on a sentence
def wsdword(text):
    match = re.search(r'<WSD>(.*?)</WSD>', text)
    if match:
        word_inside_wsd = match.group(1)
        return word_inside_wsd

In [11]:
#Funtion to counts the number of tokens : input and output tokens
from nltk.tokenize import word_tokenize

def count_tokens(text):
    # Tokenize the input text
    tokens = word_tokenize(text)
    
    # Count the tokens in the input
    token_count = len(tokens)
    
    return token_count

In [12]:
#Word lemmatizer to get the base word of the WSD word
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\2358452\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
import json
def load_dictionary_from_file(filename):
    try:
        with open(filename, 'r') as file:
            data_dict = json.load(file)
        return data_dict
    except FileNotFoundError:
        # Handle the case where the file doesn't exist
        return None
    except json.JSONDecodeError:
        # Handle the case where the file contains invalid JSON
        return None

In [14]:
def postag_preprocess(pos_tag):
    if pos_tag=="adj":
        return "adjective"
    elif pos_tag=="noun" or pos_tag=="verb":
        return pos_tag
    else:
        return None

In [15]:
loaded_dict = load_dictionary_from_file("my_dictionary.json") 

In [34]:
def remove_elements_by_key(meaning_list, key_to_remove):
    # Find indices of elements with matching keys
    indices_to_remove = [i for i, (key, _) in enumerate(meaning_list) if key == key_to_remove]

    # Remove elements from the list in reverse order to avoid index errors
    for index in sorted(indices_to_remove, reverse=True):
        meaning_list.pop(index)
    return meaning_list

In [47]:
def convert_to_json(output):
    # Remove triple quotes and leading/trailing whitespace
    output = output.strip()[7:-3].strip()

    # Convert to JSON format
    json_data = json.loads(output)

    # Convert back to a JSON string with indentation for readability
    formatted_json = json.dumps(json_data, indent=4)

    return formatted_json

In [53]:
def sense_Tag_Return_pipeline(sentence,postag,wordwsd):
    
        ob="{"
        cb="}"
        
        meanings = retrieve_meanings(wordwsd, list_of_lists)
        #print(meanings)
        cleaned_sentence= extract_word_and_index(sentence) 
        
        #retrieving the data form the dictionary
        filtered_definitions = {key: value for key, value in meanings.items() if postag in key}
        #print(filtered_definitions)
        meaning_list= list(filtered_definitions.items())      
        #print(meaning_list)
        count=0 #variable to count the tokens 
        
        if wsdword(sentence) in loaded_dict:
            #print("examples found")
            examples=loaded_dict[wsdword(sentence)]
            try:
                examples=examples[postag]
            except:
                examples=examples
            

        elif word_base(wsdword(sentence)) in loaded_dict:
            #print("examples found")
            examples=loaded_dict[word_base(wsdword(sentence))]
            try:
                examples=examples[postag]
            except:
                examples=examples
        else:
            examples=None
            
        #print(examples)
        
        
        count=0 #variable to count the tokens 
        flag=0
        for count_meaning in range(0,len(meaning_list)):
            filtered_examples=[]
            if(examples != None):
                for example in examples:
                    for meaning in meaning_list:
                        if meaning[0] in example:
                            filtered_examples.append(example)
                    
            #print(filtered_examples)                   
            prompt_predict = f'''You are going to identify the corresponding sense tag of an ambiguous word in English sentences. Use multiple reasoning strategies to increase confidence in your answer.
1. The word "{wordwsd}" has different meanings. Below are possible meanings. Comprehend the sense tags and meanings. Synonyms are provided if available. {meaning_list}
2. You can learn more on the usage of each word and the its sense through the examples below. Each sentence is followed by its corresponding sense id. "{filtered_examples}"
3. Now carefully examine the sentence below. The ambiguous word is enclosed within <WSD>."{sentence}"
4. Analyze the sentence using the following techniques and identify the meaning of the ambiguous word.
   Focus on keywords in the sentence surrounding the ambiguous word. 
   Think about the overall topic and intent of the sentence. Decide on the sense of the word that makes the most logical sense within the context. 
5. Based on the identified meaning, try to find the most appropriate senseID from the below sense tag list. You are given definition of each sense tag too."{meaning_list}".
6. Return only the identified senseid in the below JSON format.
    {ob}"sense_id":  the identified sense id{cb} '''
            
            count+=count_tokens(prompt_predict)
            response=complete_model(prompt_predict)
            #print(response)
            try:
                data = json.loads(response)
            except:
                standard_json = convert_to_json(response)
                data = json.loads(standard_json)

            value = data['sense_id']
            try:
                identified_meaning=filtered_definitions[value]
            except:
                continue
            #print(identified_meaning)

            prompt_verify=f'''You are going to verify the corresponding sense tag of an ambiguous word in English sentences. Use multiple reasoning strategies to increase confidence in your answer.
1  Carefully examine the sentence below. The ambiguous word is enclosed within <WSD>."{sentence}"
2. Analyze the sentence using the following techniques and identify the meaning of the ambiguous word.
   Focus on keywords in the sentence surrounding the ambiguous word. 
   Think about the overall topic and intent of the sentence.
3. Verify whether the meaning you identified is same as below meaning or not. {identified_meaning}
4. If same return TRUE else Return FALSE
4. Return only the TRUE or FALSE in the below JSON format.
    {ob}"Response":  TRUE/FALSE{cb} '''
            
            count+=count_tokens(prompt_verify)
            response_verify=complete_model(prompt_verify)
            #print(response_verify)

            if ("TRUE" in response_verify):
                flag=1
                #print(response)
                return value,count
            else:
                meaning_list=remove_elements_by_key(meaning_list,value)
                #print(meaning_list)
            
        if flag==0:
            return None,count     
       

        

In [54]:
import datetime
#evaluating the results
filename="missing_data_analysis/predictnverifyv2"+datetime.date.today().strftime("%B %d, %Y")+".txt"
file=open("missing_data_analysis/missing_test_data_new.txt","r",encoding="utf8")

file2=open(filename,"w",encoding="utf8")
totalTokens=0
for i in file:
    lst=i.split("	")
    sentence,senseid=lst[0],lst[1]
    postag=senseid.split(".")[1]  #postag 
    wordfromtext=senseid.split(".")[0] #wsdword
    #print(postag)
    foutput,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
    totalTokens+=token_count
    #print(output)
    print(foutput)
    #writing the output in the file
    file2.write(str(foutput) + "\n")
    

file.close()
file2.close()
print(totalTokens)

corbeau.noun.0
beast.noun.0
brake.noun.17
godship.noun.1
eyeballer.noun.2
stage.noun.11
part.noun.11
lord.noun.3
red_shirt.noun.0
glamourama.noun.0
None
corruption.noun.0
magic.noun.0
concretization.noun.1
bourne.noun.1
egg_crating.noun.2
concept.noun.1
farming.noun.1
naturalism.noun.4
moral_authority.noun.0
ergative.noun.1
fifth_wall.noun.2
patmosian.noun.1
coryphaeus.noun.0
mean.noun.0
light_of_one's_eye.noun.0
preface.noun.0
sketch.noun.1
rinky-dink.noun.0
eggcrate.noun.5
nutso.noun.0
loo.noun.8
delicate.noun.1
none.noun.0
snare.noun.0
sociation.noun.0
odd_job.noun.1
venire.noun.0
pubbie.noun.2
pursiness.noun.0
post.noun.11
nature.noun.3
wyrd.noun.0
coatful.noun.0
swoose.noun.1
pull_time.noun.2
foo.noun.2
inverted_circumflex.noun.1
castellany.noun.1
arctic_bear.noun.2
mercenary.noun.1
incivility.noun.0
green_belt.noun.2
earl.noun.1
hand.noun.11
interferant.noun.1
appearance.noun.3
knightdom.noun.1
discord.noun.0
turn_of_foot.noun.2
part.noun.14
sundowner.noun.0
grievance.noun.2
ditt

# 68 instances are corrected using this apporach